In [1]:
from transformers import AutoModel, AutoTokenizer
import torch
import json
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
model = AutoModel.from_pretrained(
    'mesolitica/llama2-embedding-2b-8k-contrastive',
    use_flash_attention_2 = True, 
    torch_dtype = torch.bfloat16,
    trust_remote_code = True,
)

A new version of the following files was downloaded from https://huggingface.co/mesolitica/llama2-embedding-2b-8k-contrastive:
- modeling_contrastive.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [3]:
_ = model.cuda()

In [4]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-2b-8k-contrastive')

In [5]:
contexts, questions = [], []

with open('/home/ubuntu/lom-agc/ultrachat-lom-agc.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if l[1]['content_ms']:
            contexts.append(l[0]['content'].strip())
            questions.append(l[1]['content_ms'].strip())
            
len(contexts), len(questions)

(8034, 8034)

In [6]:
contexts_v = []
for i in tqdm(range(len(contexts))):
    input_ids = tokenizer([contexts[i]], 
        return_tensors = 'pt',
        padding = True
    )
    v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
    contexts_v.append(v)

100%|██████████| 8034/8034 [11:22<00:00, 11.77it/s]


In [7]:
questions_v = []
for i in tqdm(range(len(contexts_v))):
    input_ids = tokenizer([questions[i]], 
        return_tensors = 'pt',
        padding = True
    )
    v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
    questions_v.append(v)

100%|██████████| 8034/8034 [02:04<00:00, 64.55it/s]


In [8]:
contexts_v_np = np.array(contexts_v)[:,0]
questions_v_np = np.array(questions_v)[:,0]

In [9]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}

for i in tqdm(range(len(questions_v_np))):
    argsort = np.argsort(cosine_similarity(questions_v_np[i].reshape(1, -1), contexts_v_np)[0])[::-1]
    for k in tops.keys():
        if i in argsort[:k]:
            tops[k] += 1

100%|██████████| 8034/8034 [03:52<00:00, 34.57it/s]


In [10]:
np.argsort(cosine_similarity(questions_v_np[i].reshape(1, -1), contexts_v_np)[0])[::-1][:10]

array([ 959,  957,  954,  950,  961,  952,  955, 3585,  944,  951])

In [12]:
for k, v in tops.items():
    print(k, v / len(questions_v_np))

1 0.09658949464774708
3 0.194050286283296
5 0.25093353248693057
10 0.34615384615384615
